# 🏨 BookMeBot – AI-Powered Hotel Booking Assistant

## Intro
Imagine having a natural, human-like conversation with an AI assistant that helps you effortlessly plan your hotel stay. This assistant, BookMeBot, is designed to guide users through the entire booking process, using conversational AI and integrated tools to:
* 🗓 Ask for check-in, check-out dates, and location
* 🛏 Check room availability for the selected destination and dates
* 🧭 Suggest top-rated tourist attractions (via Google Places API)
* 🎉 Fetch local events happening during the stay (via Ticketmaster API)
* 🍽 Recommend daily complimentary dishes from a weekly menu (mock RAG)
* ✅ Confirm the booking and provide a complete summary

## About This Project
This notebook is part of the Generative AI Intensive Capstone Project (2025Q1). The goal is to demonstrate how multiple GenAI capabilities can work together in a real-world application — in this case, a smart hotel booking assistant.

BookMeBot is powered by:
* LangGraph for building structured conversational workflows
* Gemini Pro via LangChain for intelligent dialogue generation
* Custom tools and APIs for grounding responses in real-world data

## GenAI Capabilities Demonstrated
As required by the capstone, this project incorporates below GenAI features from the given list:
* Function Calling: Gemini intelligently decides when to activate tools (like checking room availability or suggesting events) using structured function calls.
* Agents: BookMeBot is implemented as a looping, modular agent using LangGraph, where each node represents a specific stage in the conversation flow.
* Retrieval-Augmented Generation (RAG): Daily complimentary dishes are suggested based on a simulated weekly menu, demonstrating how external content can be retrieved to inform responses.
* Grounding: The assistant integrates real-time data from services like Google Places and Ticketmaster, grounding its responses in live, factual information.
* Structured Output / JSON Mode: Gemini outputs structured JSON responses when invoking tools, ensuring reliable and interpretable interactions through LangChain.

## Architecture Overview
The assistant is structured as a LangGraph-based flow, where each node represents a stage in the user interaction:
* chatbot node: Gemini generates intelligent replies
* human node: Captures user input
* tools node: Executes tool-based actions (e.g., API calls)

Routing logic:
* maybe_exit: Exits the loop when the user types "quit"
* maybe_use_tool: Decides if a tool is needed based on Gemini’s response

Additional infrastructure includes:
* Secure API key handling via Kaggle Secrets
* A modular setup allowing for future integrations like vector databases, multimodal inputs (e.g., photos of rooms or meals), and real-time bookings

## Notes
* This project can easily be extended to hostel, apartment, or travel planning assistants.
* Tools like confirm_booking can later write to a real database.
* RAG + VectorDB can be plugged in for multi-modal context (e.g., menu photos, reviews).

In the following sections, we'll see how LangGraph helps structure the conversation, and how Gemini Pro understands and responds to user inputs. We'll define nodes that act like 'stages' in the assistant's brain, and route user messages based on intent – like fetching event details or confirming a room booking.

# Authors
Group name: Human-AI 

* Santosh
* Arun
* Ulka


# Important Instructions


- Some cells in this notebook are checkpoints to verify that earlier steps ran correctly. Feel free to run all cells — but if you're eager to start chatting with BookMeBot, you can skip cells marked as optional.
- Once you've run the database creation and data insertion cells, the data will persist for the rest of your session.
- This app uses Python’s input() function to collect user input. To allow "Run all" without interruption, those lines are commented out by default. If you'd like to interact with the bot manually, uncomment the .invoke(...) calls.
- If you're planning to save and share the notebook using "Save & Run All", make sure to re-comment those input lines to keep it fully automated.

# Blogpost
Hyperlink Placeholer

# Youtube video
Hyperlink Placeholer

# Setup (Import & Installation)
To begin, we’ll prepare our environment by installing required packages and resolving any conflicts in the Kaggle notebook runtime. Think of this like setting up your AI's workshop.

- We install:
  * `google-genai`: Gemini SDK for GenAI interaction
  * `langgraph`, `langgraph-prebuilt`: To build the stateful agent
  * `langchain-google-genai`: LangChain connector for Gemini
- We remove a few preinstalled Kaggle packages that might cause compatibility issues.
- We also import basic libraries like NumPy and pandas, and confirm the Python version.

Run the debug code to confirm that the import & installation was successful.

In [1]:
#We import standard Python libraries used for data operations and system checks.
import numpy as np  # numerical computing
import pandas as pd  # data manipulation
import os
import sys

# List any data files available (optional)
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [2]:
# We remove pre-installed packages in the Kaggle environment (like spacy, fastai, etc.) that may conflict with the libraries we need for our GenAI assistant.
!pip uninstall -qqy jupyterlab libpysal thinc spacy fastai ydata-profiling google-cloud-bigquery google-generativeai

In [3]:
# We install the core langgraph and the packages needed for building our assistant
!pip install -qU 'langgraph==0.3.21' 'langchain-google-genai==2.1.2' 'langgraph-prebuilt==0.1.7' --no-deps
!pip install -U -q "google-genai==1.7.0"

In [4]:
# Debug Code for Install + Import confirmation
# We confirm the Python environment and SDK version, and print a success message to verify everything's working.
from google import genai
from google.genai import types

# Check installed versions
print("Python version:", sys.version)
print("Gemini SDK version:", genai.__version__)

# Final check
print("✅ Install + Import successful!")

Python version: 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]
Gemini SDK version: 1.7.0
✅ Install + Import successful!


###  Adding Retry Logic for Gemini API Calls
Sometimes, when using external APIs like Gemini, you might hit rate limits (too many requests in a short time) or experience temporary service issues. Instead of crashing the app or showing an error, we can automatically retry the failed request — just like how a person might refresh a page when it doesn't load the first time.
In below step, we define a retry policy using Google’s built-in retry utility:
* It retries the request if we receive error codes 429 (Too Many Requests) or 503 (Service Unavailable).
* This is especially useful when the assistant performs multiple back-to-back queries for complex user requests.


In [5]:
# Adds automatic retry handling for Gemini API calls to prevent failures from rate limits or temporary service issues.
from google.api_core import retry
from google import genai

# Retry on 429 (Too Many Requests) and 503 (Service Unavailable)
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

# Only wrap once to avoid stacking retries if cell is re-run
if not hasattr(genai.models.Models.generate_content, '__wrapped__'):
    genai.models.Models.generate_content = retry.Retry(predicate=is_retriable)(
        genai.models.Models.generate_content)

print("✅ 🔁 Gemini retry policy added.")


✅ 🔁 Gemini retry policy added.


# Set Up Your API Keys (via Kaggle Secrets)
To connect with Gemini and fetch real-world data from Google Places and Ticketmaster, we need to securely load our API keys.

In this step, we retrieve three keys stored in Kaggle Secrets:
* GOOGLE_API_KEY – for using Gemini via LangChain
* GOOGLE_PLACES_API_KEY – to suggest top tourist attractions
* TICKETMASTER_API_KEY – to find upcoming local events

These secrets are safely stored and injected at runtime, so no sensitive information is exposed in the notebook.
Once loaded, we set GOOGLE_API_KEY as an environment variable so that both Gemini's SDK and LangChain can detect it automatically.

✅ If everything works, you'll see a confirmation message.

In [6]:
# Load your Google API key from Kaggle secrets
from kaggle_secrets import UserSecretsClient
GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
GOOGLE_PLACES_API_KEY = UserSecretsClient().get_secret("GOOGLE_PLACES_API_KEY")
TICKETMASTER_API_KEY = UserSecretsClient().get_secret("TICKETMASTER_API_KEY")
# Set it as an environment variable so LangChain can auto-detect
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

# Debug code for Google API Load status
print("✅ API Keys loaded successfully!")

✅ API Keys loaded successfully!


 # Define Gemini LLM with LangChain

We start by setting up the LLM (Large Language Model) using the ChatGoogleGenerativeAI wrapper from LangChain. This connects our assistant to Gemini — specifically, the gemini-1.5-flash-latest model, which is fast, reliable, and ideal for this conversational use case.

This model is the “brain” of BookMeBot. It handles:
* Understanding user intent (e.g., “I want to book a hotel in Tokyo”)
* Deciding when to call tools (like checking availability or finding events)
* Tracking the full conversation context, so it doesn’t forget what the user said earlier

In [7]:
# Install all required missing dependencies
!pip install -q filetype
!pip install -qU "google-ai-generativelanguage>=0.6.16" "langchain-core>=0.3.49"
!pip install -qU "langgraph-checkpoint>=2.0.10,<3.0.0" "langgraph-sdk>=0.1.42,<0.2.0"

In [8]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest", temperature=0.4)
print("✅ Gemini model is ready!")

✅ Gemini model is ready!


# Define the Initial Chat State (HotelState)
In LangGraph, everything flows through a shared state dictionary. This is how different parts of your app (like the chatbot node or a tool call) communicate with each other.

We define a HotelState using TypedDict, which includes:
* messages: Stores the full conversation history between the user and the bot
* booking_info: Holds user-specific info like check-in dates and destination
* finished: A flag that tells LangGraph when to end the conversation

We also define:
* A system instruction (HOTELBOT_SYSINT) that tells Gemini how to behave
* A welcome message to greet the user when the chat starts

In [9]:
# Imports for LangGraph and typing
from typing_extensions import TypedDict
from typing import Annotated
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage  # Stronger type hinting for chat history

# Gemini's system behavior instruction
HOTELBOT_SYSINT = (
    "system",
    "You are HotelBot, a helpful hotel booking assistant. "
    "Greet the user and ask for their check-in date, check-out date, and location. "
    "Once received, use tools to check room availability, suggest tourist spots, suggest upcoming events, "
    "and provide a meal plan for the stay using the `get_menu_plan` tool **after a booking is confirmed**, "
    "or if the user requests meals."
)

# First message shown to the user
WELCOME_MSG = "👋 Welcome to BookMeBot! Please share your check-in date, check-out date, and the location you'd like to stay in."

# LangGraph shared memory structure (state)
class HotelState(TypedDict):
    # Chat messages exchanged during conversation
    messages: Annotated[list[BaseMessage], add_messages]

    # Info collected for booking (dates, location, etc.)
    booking_info: dict

    # Flag to indicate when the assistant should stop
    finished: bool

In [10]:
# Debug code for HotelState
def debug_hotel_state(state: dict):
    print("\n--- HotelState Debug ---")

    # Display chat messages
    print("\nChat History:")
    messages = state.get("messages", [])
    if not messages:
        print("  (No messages yet)")
    else:
        for msg in messages:
            role = getattr(msg, "type", "unknown")
            content = getattr(msg, "content", "")
            print(f"  [{role.upper()}] {content}")

    # Show booking details
    print("\nBooking Info:")
    booking_info = state.get("booking_info", {})
    if booking_info:
        for k, v in booking_info.items():
            print(f"  {k}: {v}")
    else:
        print("  (No booking details yet)")

    # Completion flag
    print("\n Finished:", state.get("finished", False))
    print("✅ State debug complete.\n")


# Create a test HotelState
from langchain_core.messages import AIMessage, HumanMessage

test_state = {
    "messages": [
        HumanMessage(content="I'd like to book a hotel in Paris."),
        AIMessage(content="Sure! What's your check-in and check-out date?")
    ],
    "booking_info": {
        "location": "Paris",
        "check_in": "2025-05-01",
        "check_out": "2025-05-05"
    },
    "finished": False
}

# Run the debug
debug_hotel_state(test_state)



--- HotelState Debug ---

Chat History:
  [HUMAN] I'd like to book a hotel in Paris.
  [AI] Sure! What's your check-in and check-out date?

Booking Info:
  location: Paris
  check_in: 2025-05-01
  check_out: 2025-05-05

 Finished: False
✅ State debug complete.



# Database Creation + Queries

To power our BookMeBot assistant with realistic data, we'll create a simple hotel booking database using SQLite – a lightweight, file-based database perfect for notebooks.

In this section, we'll:
* Create the database schema – with tables like customer, room, orders, booking, and location.
* Insert sample records – including test rooms, customers, and bookings so the assistant has something to work with.
* Define a helpful view – to easily query booking durations and locations.
* Write Python helper functions – to fetch data and test our setup using SQL from code.

This database will be used throughout the rest of the notebook whenever BookMeBot needs to check room availability, retrieve booking details, or respond with personalized information.

In [11]:
# Load the SQL extension
%load_ext sql

# Connect to a SQLite database (this will create 'hotel.db' in the current directory)
%sql sqlite:///hotel.db

In [12]:
%%sql

-- Create the 'location' table
CREATE TABLE IF NOT EXISTS location (
    location_id INTEGER PRIMARY KEY AUTOINCREMENT,
    location_number INTEGER NOT NULL,
    location_name VARCHAR(255) NOT NULL
);

-- Create the 'customer' table
CREATE TABLE IF NOT EXISTS customer (
  	customer_id INTEGER PRIMARY KEY AUTOINCREMENT,
  	Customer_name VARCHAR(255) NOT NULL,
    customer_number INTEGER NOT NULL,
  	order_id INTEGER,
    FOREIGN KEY (order_id) REFERENCES orders (order_id)
);

-- Create the 'orders' table
CREATE TABLE IF NOT EXISTS orders (
    order_id INTEGER PRIMARY KEY AUTOINCREMENT,
    order_number INTEGER NOT NULL,
    customer_id INTEGER NOT NULL,
    room_id INTEGER NOT NULL,
    location_id INTEGER NOT NULL,
    check_in_date TEXT NOT NULL,
    check_out_date TEXT NOT NULL,
    Order_note VARCHAR(255),
    FOREIGN KEY (customer_id) REFERENCES customer (customer_id),
    FOREIGN KEY (room_id) REFERENCES room (room_id),
    FOREIGN KEY (location_id) REFERENCES location (location_id)
);

-- Create the 'room' table
CREATE TABLE IF NOT EXISTS room (
    room_id INTEGER PRIMARY KEY AUTOINCREMENT,
    room_number INTEGER NOT NULL,
    room_booked BOOLEAN NOT NULL,
    check_in_date TEXT NOT NULL,
    check_out_date TEXT NOT NULL,
    location_id INTEGER NOT NULL,
    order_id INTEGER,
    FOREIGN KEY (order_id) REFERENCES orders (order_id),
    FOREIGN KEY (location_id) REFERENCES location (location_id)
);

-- Create the 'booking' table
CREATE TABLE IF NOT EXISTS booking (
    booking_id INTEGER PRIMARY KEY AUTOINCREMENT,
    booking_number INTEGER NOT NULL,
    room_id INTEGER NOT NULL,
    customer_id INTEGER NOT NULL,  
    location_id INTEGER NOT NULL,    
    book_date_in TEXT NOT NULL,
    book_date_out TEXT NOT NULL,
    FOREIGN KEY (room_id) REFERENCES room (room_id),
    FOREIGN KEY (customer_id) REFERENCES customer (customer_id),
    FOREIGN KEY (location_id) REFERENCES location (location_id)
);

 * sqlite:///hotel.db
Done.
Done.
Done.
Done.
Done.


[]

In [13]:
%%sql

-- Drop the view if it already exists
DROP VIEW IF EXISTS order_location;

-- Create the view joining orders with location and calculating number of days
CREATE VIEW IF NOT EXISTS order_location AS 
SELECT 
    location.location_id,
    location.location_name,
    order_number,
    order_id,
    check_in_date,
    check_out_date,
    CAST(
        JulianDay(check_out_date) - JulianDay(check_in_date)
    AS INTEGER) AS no_of_days
FROM orders
INNER JOIN location 
    ON location.location_id = orders.location_id
ORDER BY order_id DESC;

 * sqlite:///hotel.db
Done.
Done.


[]

In [14]:
%%sql

-- Insert into 'location' table
INSERT INTO location (location_number, location_name) VALUES
    (1, 'Liverpool'),
    (2, 'New York'),
    (3, 'London');

-- Insert into 'customer' table
INSERT INTO customer (customer_name, customer_number, order_id) VALUES
    ('Mr Superman', 1, 1),
    ('Ms Wonderwoman', 2, 2),
    ('Mr Spiderman', 3, 3);

-- Insert into 'orders' table
INSERT INTO orders (order_number, customer_id, room_id, location_id, check_in_date, check_out_date, order_note) VALUES
    (1, 1, 101, 1, '2025-04-01', '2025-04-15', 'Need transfer to hotel from airport upon arrival: Please arrange'),
    (2, 2, 102, 2, '2025-04-01', '2025-04-15', 'I am a vegetarian: Please arrange accordingly'),
    (3, 3, 103, 3, '2025-04-01', '2025-04-15', 'Need transfer to hotel from airport upon arrival: Please arrange'),
    (4, 2, 104, 1, '2025-04-01', '2025-04-15', 'Need transfer to hotel from airport upon arrival: Please arrange');

-- Insert into 'room' table
INSERT INTO room (room_number, room_booked, check_in_date, check_out_date, location_id) VALUES
    (101, 1, '2025-04-01', '2025-04-30', 1),
    (102, 1, '2025-04-01', '2025-04-07', 1),
    (103, 1, '2025-04-01', '2025-04-15', 1),
    (104, 1, '2025-04-17', '2025-04-30', 1),
    (105, 1, '2025-04-01', '2025-04-30', 1),
    (201, 1, '2025-04-01', '2025-04-30', 2),
    (202, 1, '2025-04-01', '2025-04-30', 2),
    (203, 1, '2025-04-01', '2025-04-30', 2),
    (204, 1, '2025-04-01', '2025-04-30', 2),
    (205, 1, '2025-04-01', '2025-04-30', 2),
    (301, 1, '2025-04-01', '2025-04-30', 3),
    (302, 1, '2025-04-01', '2025-04-30', 3),
    (303, 1, '2025-04-01', '2025-04-20', 3),
    (304, 1, '2025-04-01', '2025-04-30', 3),
    (305, 1, '2025-04-01', '2025-04-30', 3);


 * sqlite:///hotel.db
3 rows affected.
3 rows affected.
4 rows affected.
15 rows affected.


[]

In [15]:
%%sql

-- Insert realistic booking data for BookMeBot simulation
INSERT INTO booking (
    booking_number,
    room_id,
    customer_id,
    location_id,
    book_date_in,
    book_date_out
)
VALUES 
    (1, 102, 1, 1, '2025-04-08', '2025-04-10'),
    (2, 103, 2, 2, '2025-04-11', '2025-05-14'),
    (3, 104, 3, 3, '2025-04-12', '2025-04-18');


 * sqlite:///hotel.db
3 rows affected.


[]

In [16]:
# We connect to the database from Python using sqlite3 so we can run SQL queries from code, not just SQL cells.

import sqlite3

db_file = "hotel.db"
db_conn = sqlite3.connect(db_file)

### Optional Utility Functions for Tables, Views, and Queries
These helper functions allow you to:

* List all tables (list_tables)
* List views like order_location (list_views)
* Run SQL queries and fetch results (execute_query)

In [17]:
# Optional cell
def list_tables() -> list[str]:
    """Retrieve the names of all tables in the database."""
    # Include print logging statements so you can see when functions are being called.
    print(' - DB CALL: list_tables()')

    cursor = db_conn.cursor()

    # Fetch the table names.
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

    tables = cursor.fetchall()
    return [t[0] for t in tables]

list_tables()

 - DB CALL: list_tables()


['location', 'sqlite_sequence', 'customer', 'orders', 'room', 'booking']

In [18]:
# Optional cell
def list_views() -> list[str]:
    """Retrieve the names of all views in the database."""
    # Include print logging statements so you can see when functions are being called.
    print(' - DB CALL: list_views()')

    cursor = db_conn.cursor()

    # Fetch the table names.
    cursor.execute("SELECT name FROM sqlite_master WHERE type='view';")

    views = cursor.fetchall()
    return [t[0] for t in views]

list_views()

 - DB CALL: list_views()


['order_location']

In [19]:
def execute_query(sql: str) -> list[str]:
    """Execute an SQL statement, returning the results."""
    ## print(f' - DB CALL: execute_query({sql})')

    cursor = db_conn.cursor()

    cursor.execute(sql)
    return cursor.fetchall()

### Test the Queries

These lines test your connection and query logic by fetching:

* All locations
* All customers
* Room info
* Booking durations via order_location view


In [20]:
# Optional cell
## execute_query ("select * from location")
##execute_query ("select * from customer")
##execute_query ("select * from room")
execute_query ("select * from booking")

[(1, 1, 102, 1, 1, '2025-04-08', '2025-04-10'),
 (2, 2, 103, 2, 2, '2025-04-11', '2025-05-14'),
 (3, 3, 104, 3, 3, '2025-04-12', '2025-04-18'),
 (4, 999, 11, 1, 3, 'May 1st', 'May 5th'),
 (5, 999, 11, 1, 3, '2025-05-10', '2025-05-14'),
 (6, 999, 12, 1, 3, 'May 1st', 'May 5th'),
 (7, 999, 12, 1, 3, '2025-05-10', '2025-05-14'),
 (8, 999, 13, 1, 3, 'May 1st', 'May 5th'),
 (9, 1, 102, 1, 1, '2025-04-08', '2025-04-10'),
 (10, 2, 103, 2, 2, '2025-04-11', '2025-05-14'),
 (11, 3, 104, 3, 3, '2025-04-12', '2025-04-18'),
 (12, 999, 13, 1, 3, '2025-05-10', '2025-05-14'),
 (13, 999, 14, 1, 3, '2025-05-10', '2025-05-14'),
 (14, 999, 11, 1, 3, 'April 1st', 'April 5th'),
 (15, 999, 11, 1, 3, 'July 1st', 'July 5th'),
 (16, 999, 11, 1, 3, 'June 1st', 'June 5th'),
 (17, 999, 15, 1, 3, '2025-05-10', '2025-05-14'),
 (18, 999, 12, 1, 3, 'April 1st', 'April 5th'),
 (19, 1, 102, 1, 1, '2025-04-08', '2025-04-10'),
 (20, 2, 103, 2, 2, '2025-04-11', '2025-05-14'),
 (21, 3, 104, 3, 3, '2025-04-12', '2025-04-18')

In [21]:
# Sets up a new SQLite connection that returns cleaner, single-column query results by default instead of tuples. 

db_conn.close()
db_file = "hotel.db"
conn = sqlite3.connect(db_file)
conn.row_factory = lambda cursor, row: row[0]

In [22]:
def execute_query(sql: str) -> list[str]:
    """Execute an SQL statement, returning the results."""
    ## print(f' - DB CALL: execute_query({sql})')

    cursor = conn.cursor()

    cursor.execute(sql)
    return cursor.fetchone()

# Step-by-Step: Define BookMeBot Tools
In this section, we define a set of tools that give BookMeBot real-world capabilities beyond simple chat.

These tools are like skills the assistant can use when needed, such as:
* Checking room availability
* Finding top tourist attractions using the Google Places API
* Looking up upcoming events via Ticketmaster
* Recommending complimentary meals
* Confirming bookings (simulated or real database insertions)

Each tool is defined as a Python function and wrapped using LangChain’s @tool decorator, allowing Gemini to dynamically call them during a conversation based on user input.

At the end, we bundle all tools and connect them to the Gemini LLM, enabling a truly interactive and smart assistant.

In [23]:
def normalize_date(date_str: str) -> str:
    """
    Converts natural date input to ISO format (YYYY-MM-DD).
    Returns None if parsing fails.
    """
    try:
        return parse(date_str).strftime("%Y-%m-%d")
    except Exception:
        return None


### 1. check_room_availability()
A basic tool that simulates room availability for popular cities. Currently hardcoded, but designed to later integrate with your database.

In [24]:
from langchain_core.tools import tool

@tool
def check_room_availability(location: str, check_in: str, check_out: str) -> str:
    """
    Simulates checking room availability in a hotel.
    Later, replace this with a real database query.
    """
    print(f"🔍 Checking rooms in {location} from {check_in} to {check_out}...")
    if location.lower() in ["london", "liverpool", "new york"]:
        return f"✅ Rooms available in {location} from {check_in} to {check_out}."
    else:
        return f"❌ Sorry, we do not have properties in {location}."

In [25]:
# ✅ Debug Code: Manually test check_room_availability tool
check_room_availability.invoke({
    "location": "London",
    "check_in": "2025-05-01",
    "check_out": "2025-05-05"
})

🔍 Checking rooms in London from 2025-05-01 to 2025-05-05...


'✅ Rooms available in London from 2025-05-01 to 2025-05-05.'

### Once a booking is confirmed, it becomes a CONFIRMED order for our hotel! 

So, we will take that latest confirmed order and display the tourist spots and events close to that location/date in the following section.

### 2. get_tourist_spots(location)
This tool uses the Google Places API to fetch and format the top 5 tourist attractions in the customer’s destination city.

In [26]:
import requests  # This library helps you make HTTP API requests

# ✅ Tool: Get Top Tourist Spots (Tool-Ready Version)
@tool
def get_tourist_spots(location: str) -> str:
    """
    Fetches and formats the top 5 tourist attractions using Google Places API.
    
    Parameters:
    - location (str): City name (e.g., 'London', 'New York')

    Returns:
    - str: Formatted string listing top 5 attractions
    """
    # print(f"📍 Fetching tourist spots for: {location}...")

    url = "https://maps.googleapis.com/maps/api/place/textsearch/json"
    params = {
        "query": f"top tourist attractions in {location}",
        "key": GOOGLE_PLACES_API_KEY
    }

    try:
        response = requests.get(url, params=params)
        data = response.json()
    except Exception as e:
        return f"❌ Error contacting Google Places API: {e}"

    if data.get("status") != "OK":
        return f"❌ API returned: {data.get('error_message', 'Unknown error')}"

    results = data.get("results", [])[:5]
    if not results:
        return f"ℹ️ No popular attractions found in {location}."

    output = f"🏞️ Top Tourist Spots in {location.title()}:\n"
    for i, place in enumerate(results, 1):
        name = place.get("name")
        address = place.get("formatted_address", "N/A")
        rating = place.get("rating", "N/A")
        output += f"{i}. {name} (⭐ {rating})\n   📍 {address}\n"

    return output


In [27]:
# Debug Code to test the function
get_tourist_spots("London")

/tmp/ipykernel_324/24624846.py:2: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  get_tourist_spots("London")


'🏞️ Top Tourist Spots in London:\n1. The View from The Shard (⭐ 4.6)\n   📍 32 London Bridge St, London SE1 9SG, United Kingdom\n2. Tower of London (⭐ 4.7)\n   📍 London EC3N 4AB, United Kingdom\n3. The London Dungeon (⭐ 4.3)\n   📍 Riverside Building, County Hall, Westminster Bridge Rd, London SE1 7PB, United Kingdom\n4. lastminute.com London Eye (⭐ 4.5)\n   📍 Riverside Building, County Hall, London SE1 7PB, United Kingdom\n5. SEA LIFE London Aquarium (⭐ 4.3)\n   📍 Riverside Building, County Hall, Westminster Bridge Rd, London SE1 7PB, United Kingdom\n'

### 3. get_events(location, event_date, days)
This tool connects to the Ticketmaster API and returns upcoming events based on the check-in date and location.

In [28]:
from langchain_core.tools import tool
import requests
from kaggle_secrets import UserSecretsClient

# Load your Ticketmaster API Key from Kaggle Secrets
secrets = UserSecretsClient()
TICKETMASTER_API_KEY = secrets.get_secret("TICKETMASTER_API_KEY")

@tool
def get_events(location: str, event_date: str = "2025-05-01", days: int = 5) -> str:
    """
    Fetches upcoming events using Ticketmaster API and formats them for display.
    """
    print(f"🎫 Searching events in '{location}' on or after {event_date}...")

    url = "https://app.ticketmaster.com/discovery/v2/events.json"
    params = {
        "apikey": TICKETMASTER_API_KEY,
        "city": location,
        "startDateTime": f"{event_date}T00:00:00Z",
        "size": days,
        "sort": "date,asc"
    }

    try:
        response = requests.get(url, params=params)
        data = response.json()
    except Exception as e:
        return f"❌ Error connecting to Ticketmaster: {str(e)}"

    if "_embedded" not in data:
        return "📭 No events found or something went wrong."

    output = f"🎟️ Events in {location} after {event_date}:\n"
    for e in data["_embedded"]["events"]:
        name = e["name"]
        venue = e["_embedded"]["venues"][0]["name"]
        date = e["dates"]["start"]["localDate"]
        output += f"* {date}: {name} at {venue}\n"

    return output


In [29]:
# Debug Code: pretty print results
#results = get_events.invoke({
#    "location": location,
#    "event_date": datein,
#    "days": noofdays
#})

#print(results)


### Daily Menu Suggestion (Mock RAG)
Returns a static 7-day complimentary menu based on location and dates.

In [30]:
from datetime import datetime, timedelta
from dateutil.parser import parse
from langchain_core.tools import tool

@tool
def get_menu_plan(location: str, check_in: str, check_out: str) -> str:
    """
    Generates a simple rotating meal plan based on the stay duration.
    Accepts check-in and check-out in flexible formats (e.g., 'May 1st').
    """
    daily_dishes = [
        "Paneer Butter Masala",
        "Chole Bhature",
        "Rajma Chawal",
        "Veg Biryani",
        "Dal Baati Churma",
        "Aloo Paratha",
        "Masala Dosa"
    ]

    try:
        # Normalize date inputs
        start_date = parse(check_in).date()
        end_date = parse(check_out).date()
    except Exception as e:
        return f"❌ Could not understand the dates provided. Please use a valid format like '2025-05-01'. Error: {str(e)}"

    num_days = (end_date - start_date).days
    if num_days <= 0:
        return "❌ Invalid stay duration. Check-out must be after check-in."

    output = f"📅 Daily Menu for {location} from {start_date} to {end_date}:\n"
    for i in range(num_days):
        date = (start_date + timedelta(days=i)).strftime("%Y-%m-%d")
        dish = daily_dishes[i % len(daily_dishes)]
        output += f"👉 {date}: {dish}\n"

    output += "\nPlease choose one dish per day as your complimentary meal 😊"
    return output

In [31]:
# 🧪 Debug Function for Menu Plan
def debug_menu_plan(location: str, check_in: str, check_out: str):
    """
    Tests the get_menu_plan tool with sample input.
    Automatically detects whether it's decorated with @tool or not.
    """
    try:
        # If tool-decorated, use .invoke()
        result = get_menu_plan.invoke({
            "location": location,
            "check_in": check_in,
            "check_out": check_out
        })
    except AttributeError:
        # Fall back to direct call if not tool-decorated
        result = get_menu_plan(location, check_in, check_out)

    print(result)
debug_menu_plan("New York", "2025-04-20", "2025-04-25")


📅 Daily Menu for New York from 2025-04-20 to 2025-04-25:
👉 2025-04-20: Paneer Butter Masala
👉 2025-04-21: Chole Bhature
👉 2025-04-22: Rajma Chawal
👉 2025-04-23: Veg Biryani
👉 2025-04-24: Dal Baati Churma

Please choose one dish per day as your complimentary meal 😊


### Booking Confirmation Tool
This section defines the tool for confirming hotel bookings that performs a real booking by inserting into the database and updating room availability.

In [32]:
from langchain_core.tools import tool
import sqlite3

@tool
def confirm_booking_db(location: str, check_in: str, check_out: str) -> str:
    """
    Confirms the booking by finding an available room and saving it in the booking table.
    """
    print(f"Booking in {location} from {check_in} to {check_out}...")

    db_file = "hotel.db"
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    # Step 1: Get location ID
    cursor.execute("SELECT location_id FROM location WHERE location_name = ?", (location,))
    loc_row = cursor.fetchone()
    if not loc_row:
        return f"❌ Location '{location}' not found."
    location_id = loc_row[0]

    # Step 2: Find available room (based on date range, not just room_booked)
    cursor.execute("""
        SELECT room_id FROM room
        WHERE location_id = ?
        AND room_id NOT IN (
            SELECT room_id FROM booking
            WHERE NOT (
                book_date_out <= ? OR book_date_in >= ?
            )
        )
        LIMIT 1
    """, (location_id, check_in, check_out))
    room_row = cursor.fetchone()

    if not room_row:
        return f"❌ No available rooms in {location} for the selected dates."

    room_id = room_row[0]

    # Step 3: Insert into booking table
    cursor.execute("""
        INSERT INTO booking (booking_number, room_id, customer_id, location_id, book_date_in, book_date_out)
        VALUES (?, ?, ?, ?, ?, ?)
    """, (999, room_id, 1, location_id, check_in, check_out))  # booking_number = 999 for demo, customer_id = 1

    conn.commit()
    conn.close()

    return (
        f"✅ Booking Confirmed!\n"
        f"• Location: {location}\n"
        f"• Room ID: {room_id}\n"
        f"• Dates: {check_in} to {check_out}\n"
        f". Congrats you got the best deal for 50 dollars per day. Please pay directly at the hotel during your stay\n"
        f"🎉 We look forward to hosting you!"
    )


In [33]:
# 🧪 Debug: Test the real booking tool (writes to database)
def debug_confirm_booking_db():
    location = "London"            # Use a location that exists in our `location` table
    check_in = "2025-05-10"
    check_out = "2025-05-14"

    try:
        # If tool-decorated, use invoke (LangChain-style)
        result = confirm_booking_db.invoke({
            "location": location,
            "check_in": check_in,
            "check_out": check_out
        })
    except AttributeError:
        # Fallback to normal function call
        result = confirm_booking_db(location, check_in, check_out)

    print("📦 Booking Result:")
    print(result)
debug_confirm_booking_db()


Booking in London from 2025-05-10 to 2025-05-14...
📦 Booking Result:
✅ Booking Confirmed!
• Location: London
• Room ID: 26
• Dates: 2025-05-10 to 2025-05-14
. Congrats you got the best deal for 50 dollars per day. Please pay directly at the hotel during your stay
🎉 We look forward to hosting you!


# Bind Tools to Gemini & Register Them with LangGraph
Now that we’ve defined all our tools (for checking rooms, events, places, meals, and booking), at this stage we teach our assistant how to actually call tools during a conversation — using LangGraph’s ToolNode

* Tell Gemini what tools it can use (bind_tools)
* Register those tools in a ToolNode, so LangGraph knows how to call them when needed

This enables the assistant to intelligently decide:
* "Should I fetch events?"
* "Should I check availability?"
* "Should I call the Google Places API?"

In [34]:
from langgraph.prebuilt import ToolNode

# Step 1: Group all tools into one list
all_tools = [
    check_room_availability,  # Simulated room availability
    get_tourist_spots,        # Tourist spots via Google API
    get_events,               # Events via Ticketmaster API
    confirm_booking_db,          # Simulated confirmation
    get_menu_plan             # Daily meal plan suggestion
]

# Step 2: Tell Gemini about these tools
llm_with_tools = llm.bind_tools(all_tools)

# Step 3: Create a ToolNode that will be used in the LangGraph
tool_node = ToolNode(all_tools)

In [35]:
# Debug Code – Confirm tool names registered in the node
print("✅ ToolNode registered with the following tools:")
for tool_name in tool_node.tools_by_name:
    print(f"• {tool_name}")

✅ ToolNode registered with the following tools:
• check_room_availability
• get_tourist_spots
• get_events
• confirm_booking_db
• get_menu_plan


# Define the Chatbot Node (Gemini's Reply Logic)
This function is the heart of BookMeBot — it controls how the assistant thinks and responds.

Every time the user says something, this node:
* Takes the current state of the chat (conversation history, booking info, flags, etc.)
* Sends it to Gemini to generate a response, using the llm_with_tools model
* Returns the updated state, now including the assistant’s reply

Key Concepts
* state["messages"] holds the chat history.
* llm_with_tools.invoke(...) sends everything to Gemini and gets a smart reply.
* The assistant becomes context-aware, remembering what was said earlier.

This is how BookMeBot maintains a memory of the conversation and can respond smartly to follow-up questions, like:
“What’s the meal plan?” or “Can I change the check-out date?”

In [36]:
# Node that generates HotelBot replies using Gemini
def chatbot_node(state: HotelState) -> HotelState:
    # print("🔄 Chatbot node triggered.")

    # Case 1: If there’s already chat history (user has spoken before)
    if state["messages"]:
        # Combine system prompt + full conversation history
        reply = llm_with_tools.invoke([HOTELBOT_SYSINT] + state["messages"])
    else:
        # Case 2: First time user enters – show welcome message
        reply = AIMessage(content=WELCOME_MSG)

    # Update and return the new state with bot’s reply
    return state | {"messages": [reply]}


# Create the Human Node (Simulate User Input)

LangGraph is built around a loop of interactions. So far, we’ve created a chatbot_node where Gemini replies to the user — but without a way to receive the user’s next input, the assistant can’t continue.

This node simulates the human in the loop:
* Prints Gemini's latest reply to the screen
* Accepts the user's next input via input()
* Updates the conversation state with the user's message
*  Ends the loop if the user types "quit", "bye", or similar

This node forms the back-and-forth conversational cycle — human says something → Gemini replies → repeat.

In [37]:
def human_node(state: HotelState) -> HotelState:
    """
    Simulates a user in the loop using console input.
    Adds user message to conversation history.
    Ends the loop if the user types 'quit'.
    """
    # 🗨️ Show the assistant’s most recent message
    last_msg = state["messages"][-1]
    print(f"\n🤖 BookMeBot: {last_msg.content}\n")

    # ⌨️ Accept next input from the user
    user_input = input("👤 You: ")

    # 🛑 Check for exit command
    if user_input.lower() in {"quit", "exit", "bye", "q"}:
        state["finished"] = True

    # ➕ Add the new user message to the state
    return state | {"messages": [("user", user_input)]}

# Define Router Functions (Routing Logic for LangGraph)
LangGraph lets you build dynamic, branching flows using routing functions. These small functions help the chatbot decide what to do next, based on the conversation state.

In [38]:
# Decides if the chat should end. If the user said "quit", exit the conversation; otherwise, go back to Gemini.

from langgraph.graph import END
from typing import Literal

def maybe_exit(state: HotelState) -> Literal["chatbot", "__end__"]:
    # print("🧭 Routing decision (maybe_exit)...")  # Debug
    return END if state.get("finished") else "chatbot"


In [39]:
# Decides if Gemini wants to use a tool or wait for more user input. If Gemini made a tool call (like get_events), route to tools. Otherwise, continue talking to the user.
def maybe_use_tool(state: HotelState) -> Literal["tools", "human"]:
    # print("🧭 Routing decision (maybe_use_tool)...")  # Debug

    last_msg = state.get("messages", [])[-1]

    if hasattr(last_msg, "tool_calls") and len(last_msg.tool_calls) > 0:
        return "tools"
    else:
        return "human"

In [40]:
# A combined version that routes to tools, human, or exit. Exit if finished, call tool if Gemini invoked one, otherwise continue with user.
def maybe_route(state: HotelState) -> Literal["tools", "human", "__end__"]:
    if state.get("finished", False):
        return "__end__"

    last_msg = state.get("messages", [])[-1]

    if hasattr(last_msg, "tool_calls") and last_msg.tool_calls:
        # print("🛠️ Tool call detected!")
        return "tools"

    return "human"


 # Build the LangGraph — BookMeBot’s Conversation Engine
 LangGraph lets us design AI agents like flowcharts or state machines, where each node represents a step in the conversation. This makes your chatbot:

* Loopable (chat continues naturally)
* Context-aware (remembers past messages)
* Tool-augmented (calls APIs, databases, etc.)
* Decision-driven (smart routing using logic functions)

In [41]:
from typing import Literal
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode

# 🏗️ Initialize the LangGraph with the shared state schema
builder = StateGraph(HotelState)

# 🧠 Add key nodes (chatbot, human input, tools)
builder.add_node("chatbot", chatbot_node)   # Gemini replies
builder.add_node("human", human_node)       # Simulated user input
builder.add_node("tools", tool_node)        # Tool calls (API, DB, etc.)

# ➡️ Define the flow between steps

# Start at chatbot
builder.add_edge(START, "chatbot")

# After Gemini replies, route:
# → to tools if tool call detected, else → to human
builder.add_conditional_edges("chatbot", maybe_route)

# If tools were used, go back to chatbot with result
builder.add_edge("tools", "chatbot")

# After user input, check if they want to quit or continue
builder.add_conditional_edges("human", maybe_exit)

# ✅ Compile the LangGraph agent
hotel_graph = builder.compile()


In [42]:
#Debug code
print(hotel_graph.get_graph().draw_mermaid())

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	chatbot(chatbot)
	human(human)
	tools(tools)
	__end__([<p>__end__</p>]):::last
	__start__ --> chatbot;
	tools --> chatbot;
	chatbot -.-> tools;
	chatbot -.-> human;
	chatbot -.-> __end__;
	human -.-> chatbot;
	human -.-> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



# Run BookMeBot – Final Agent Loop
This is where the LangGraph-powered BookMeBot goes live. When you run this block:
* The assistant starts from the chatbot node (Gemini)
* Flows through human and tool nodes
* Uses the routing logic to loop or exit
* Keeps memory of the full conversation


⚠️ Notes
* Make sure you uncomment hotel_graph.invoke(...) to start the agent.
* The loop will continue until you type an exit word like "quit", "q", or "bye" (handled in human_node).
* The final state will show full conversation history and booking info (if tools were triggered).

Instruction: Please uncomment all the lines before the print statements to see the BookMeBot in action!

In [49]:

  #state = hotel_graph.invoke({
  # "messages": [],
  #  "booking_info": {},
  #  "finished": False
# }, config={"recursion_limit": 50})

# Optional: Print chat messages (clean format)
#for msg in state["messages"]:
#    if hasattr(msg, "type") and msg.type == "human":
#        print(f"👤 You: {msg.content}")
#    elif hasattr(msg, "type") and msg.type == "ai":
#        print(f"🤖 BookMeBot: {msg.content}")

# Descriptive goodbye
#location = state.get("booking_info", {}).get("location", "your destination")
#check_in = state.get("booking_info", {}).get("check_in", "soon")
#check_out = state.get("booking_info", {}).get("check_out", "soon")

print(f"\n👋 BookMeBot: Thanks for booking with us!")
print(f"Have a safe and wonderful trip! 🌍✈️🏨")


# 🧪 Try saying:
# - "I want to book a room in London from May 1st to May 5th"
# - "What tourist attractions are nearby?"
# - "Are there any events during my stay?"
# - "What's the meal plan?"
# - "quit" or "q" to exit the loop


🤖 HotelBot: 👋 Welcome to HotelBot! Please share your check-in date, check-out date, and the location you'd like to stay in.



👤 You:  quit



👋 HotelBot: Thanks for booking with us!
Have a safe and wonderful trip! 🌍✈️🏨
